# Compute Relative Power of EEG Bands

## Goals:
1. **Data Import**
    - Import the preprocessed data from npz and json files

2. **Feature Extraction**
    - Calculate the Power Spectral Density (PSD) for each epoch.
    - Average PSD values across all epochs 
    - Get AUC across freq values for each frequency band
    - Normalize AUC PSD values by computing the z-score for each
    
3. **Data Formatting**
    - Export the data as (1, 12, 6, 1, 16, 4)
        - (n_file, n_stimuli, (max)n_epochs, comfort_score, n_channels, n_bands)

# Import Libraries

In [1]:
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
import csv

# Custom libraries
from Functions import processing

# Import Epoched Data and Settings

In [2]:
# Load list of files to import
files = [  
    "sub-P008-Redo_ses-S001_task-T2_run-001_eeg"   
]

# Get unique subject IDs
#subject_ids = [file.split('_')[0].split('sub-')[1] for file in files]
#unique_subject_ids = list(set(subject_ids))
unique_subject_ids = ["P008"] #hardcoded for redo file

# Preallocate variables to store EEG data, settings, and comfort data
eeg_epochs = [None] * len(files)
settings = [None] * len(files)
comfort_data_dicts = [None] * len(files)

# Import data for each file
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz)
    loaded_data = np.load(f"Data\\Pilot-Data\\EEG\\sub-P008-Redo\\{file}.npz", allow_pickle=True)
    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}
    
    # Import settings
    with open(f"Data\\Pilot-Data\\EEG\\sub-P008-Redo\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)
    
    # Import comfort data"
    comfort_data = pd.read_csv(f"Data\\Pilot-Data\\EEG\\sub-P008-Redo\\{unique_subject_ids[f]}-All-Comfort.csv") 
    grouped = comfort_data.groupby(["Contrast", "Size"])
    
    # Create a dictionary for this subject: keys like "ContrastXSizeY" map to lists of comfort values
    stim_comfort_dict = {}
    for (contrast, size), group in grouped:
        key = f"Contrast{contrast}Size{size}"
        values = group["Comfort_Score"].tolist()
        stim_comfort_dict[key] = values
        
    # Save the per-subject comfort dictionary for later processing
    comfort_data_dicts[f] = stim_comfort_dict

# Compute PSD for each "On" Epoch

- Calculate AUC for every channel for 4 freq bands
    - Do not include the SSVEP stim freqs/harmonics in the frequency bands

In [3]:
# PSD settings
window_size = 10  # For 10 second epochs: 10 = 0.1 Hz resolution, 5 = 0.2 Hz resolution, 2 = 0.5 Hz resolution

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)  

# Compute PSD for each file
for f in range(len(files)):
    eeg_f[f] = {}
    eeg_pxx[f] = {}

    for stim_label, epochs in eeg_epochs[f].items():  
        eeg_f[f][stim_label] = []
        eeg_pxx[f][stim_label] = []

        for epoch in epochs:  
            f_values, pxx_values = signal.welch(
                x=epoch,  
                fs=settings[f]["eeg_srate"],
                nperseg=window_size * settings[f]["eeg_srate"],
                noverlap=(window_size * settings[f]["eeg_srate"]) * 0.5,  
            )
            eeg_f[f][stim_label].append(f_values)
            eeg_pxx[f][stim_label].append(pxx_values)

        eeg_f[f][stim_label] = np.array(eeg_f[f][stim_label]) 
        eeg_pxx[f][stim_label] = np.array(eeg_pxx[f][stim_label]) 

# Define Frequency Bands
eeg_bands = {"delta": [1, 4], "theta": [4, 8], "alpha1": [8, 9], "alpha2": [11, 13], "beta1": [13, 19], "beta2": [21, 30]}

# Preallocate AUC array: (num_files, num_stimuli, max num_epochs, num_channels, num_bands)
auc = np.zeros((len(files), len(eeg_pxx[0]), 6, len(settings[f]['new_ch_names']), 4))

# Iterate over files and stimuli
for f in range(len(files)):
    for stim_idx, stim_label in enumerate(eeg_pxx[f]):
        for epoch_idx in range(len(eeg_pxx[f][stim_label])):
            for band_idx, (band_name, band_range) in enumerate(eeg_bands.items()):
                # Fix fmask indexing for the frequency range
                fmask = (eeg_f[f][stim_label][0, :] >= band_range[0]) & (eeg_f[f][stim_label][0, :] <= band_range[1])

                # Integrate the PSD across the frequency range for each band
                if band_name == 'delta':
                    delta = np.trapezoid(eeg_pxx[f][stim_label][epoch_idx][:, fmask], x=eeg_f[f][stim_label][0, fmask], axis=1)
                    auc[f, stim_idx, epoch_idx, :, 0] = delta
                elif band_name == 'theta':
                    theta = np.trapezoid(eeg_pxx[f][stim_label][epoch_idx][:, fmask], x=eeg_f[f][stim_label][0, fmask], axis=1)
                    auc[f, stim_idx, epoch_idx, :, 1] = theta
                elif band_name == 'alpha1':
                    alpha1 = np.trapezoid(eeg_pxx[f][stim_label][epoch_idx][:, fmask], x=eeg_f[f][stim_label][0, fmask], axis=1)
                elif band_name == 'alpha2':
                    alpha2 = np.trapezoid(eeg_pxx[f][stim_label][epoch_idx][:, fmask], x=eeg_f[f][stim_label][0, fmask], axis=1)
                    full_alpha = alpha1 + alpha2
                    auc[f, stim_idx, epoch_idx, :, 2] = full_alpha
                elif band_name == 'beta1':
                    beta1 = np.trapezoid(eeg_pxx[f][stim_label][epoch_idx][:, fmask], x=eeg_f[f][stim_label][0, fmask], axis=1)
                elif band_name == 'beta2':
                    beta2 = np.trapezoid(eeg_pxx[f][stim_label][epoch_idx][:, fmask], x=eeg_f[f][stim_label][0, fmask], axis=1)
                    full_beta = beta1 + beta2
                    auc[f, stim_idx, epoch_idx, :, 3] = full_beta

# Print shape of the final AUC array
print(f"AUC: {auc.shape}") # (num_files, num_stimuli, max(num_epochs), num_channels, num_bands)


AUC: (1, 12, 6, 13, 4)


# Normalize the AUC values
- Z-score
    * First reshape so all values are one long list for each participant 
        - $(1,\ 12 \times 6 \times 16 \times 4)$ for 12 stimuli, 6 epochs, 16 channels, 4 frequency bands
    * Compute the mean and std for all AUC PSD values 
        - mean/std of 4,608 values for above example
    * Reshape the mean/std arrays so they can be combined with the original auc array
    * Calculate the z-score for each AUC PSD value
        * **Formula**:  $
\text{PSD Z-score} = \frac{\text{AUC PSD}_{\text{single channel/band}} - \mu_{\text{baseline}}}{\sigma_{\text{baseline}}}
$


In [4]:
auc_reshaped = auc.reshape(auc.shape[0], -1)
means = np.mean(auc_reshaped, axis=1, keepdims=True)
stds = np.std(auc_reshaped, axis=1, keepdims=True)

# Reshape back and compute z-score
auc_normalized = (auc - means.reshape(-1, 1, 1, 1, 1)) / stds.reshape(-1, 1, 1, 1, 1)

# Combine stimuli, comfort scores, and PSD (per freq band)

Shape will be (n_file, n_stimuli, (max)n_epochs, comfort_score, n_channels, n_bands)
- for each stim it will be arr[f,stim] = (6, 1, ~16, 4)

In [5]:
# Preallocate array with the given shape
ml_input = np.zeros((
    len(files),                        # Number of files
    len(eeg_pxx[0]),                   # Number of stimuli
    6,                                 # Max Number of Epochs
    1,                                 # Number of comfort scores per stimulus
    len(settings[f]['new_ch_names']),  # Number of channels
    4                                  # Number of bands
))  

eeg_bands = {"delta": [1, 4], "theta": [4, 8], "alpha": [8, 9], "beta": [13, 19]}

for f in range(len(files)):
    for stim_idx, stim_label in enumerate(eeg_pxx[f]):
        for epoch_idx in range(len(eeg_pxx[f][stim_label])):
            for comfort_idx, (c_stim_label, comfort_score) in enumerate(comfort_data_dicts[f].items()):
                if stim_label == c_stim_label:
                    # Get the specific comfort score for this epoch
                    comfort_value = comfort_score[epoch_idx]

                    for band_idx, (band_name, band_range) in enumerate(eeg_bands.items()):
                            # Assign the AUC value to the ml_input array
                            if band_name == 'delta':
                                ml_input[f, stim_idx, epoch_idx, 0, :, 0] = auc_normalized[f, stim_idx, epoch_idx, :, 0]
                            elif band_name == 'theta':
                                ml_input[f, stim_idx, epoch_idx, 0, :, 1] = auc_normalized[f, stim_idx, epoch_idx, :, 1]
                            elif band_name == 'alpha':
                                ml_input[f, stim_idx, epoch_idx, 0, :, 2] = auc_normalized[f, stim_idx, epoch_idx, :, 2]
                            elif band_name == 'beta':
                                ml_input[f, stim_idx, epoch_idx, 0, :, 3] = auc_normalized[f, stim_idx, epoch_idx, :, 3]

# Print shape of the ml_input array
print(f"ml_input: {ml_input.shape}")    # (num_files, num_stimuli, max(num_epochs), comfort_score(n = 1), num_channels, num_bands)

ml_input: (1, 12, 6, 1, 13, 4)


# Save ML Input to CSV

In [6]:
# Define the EEG bands and channel names
eeg_bands = {"delta": [1, 4], "theta": [4, 8], "alpha": [8, 9], "beta": [13, 19]}
ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P7", "P3", "P4", "P8", "PO7", "PO8", "O1", "Oz", "O2"]

# Define the structure for the CSV header
eeg_band_columns = [
    f"{ch}_{band}" for ch in ch_names for band in eeg_bands.keys()
]
header = ['Participant', 'Stimulus', 'Epoch', 'Comfort Score'] + eeg_band_columns

# Open the CSV file for writing
with open('Data\\Pilot-Data\\EEG\\sub-P008-Redo\\P008_norm_ml_input.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    # Iterate through the ml_input data
    for f in range(len(files)):
        # Extract participant ID from filename
        participant_id = unique_subject_ids[f]

        for stim_idx, stim_label in enumerate(eeg_pxx[f]):
            for epoch_idx in range(len(eeg_pxx[f][stim_label])):
                for comfort_idx, (c_stim_label, comfort_score) in enumerate(stim_comfort_dict.items()):
                    if stim_label == c_stim_label:
                        comfort_value = comfort_score[epoch_idx]
                        
                        row = [participant_id, stim_label, epoch_idx, comfort_value]
                        
                        channel_band_data = {f"{ch}_{band}": 0 for ch in ch_names for band in eeg_bands.keys()}
                        
                        for ch_idx, ch_name in enumerate(settings[f]['new_ch_names']):
                            for band_idx, (band_name, band_range) in enumerate(eeg_bands.items()):
                                if ch_name in ch_names:
                                    band_value = ml_input[f, stim_idx, epoch_idx, 0, ch_idx, band_idx]
                                    channel_band_data[f"{ch_name}_{band_name}"] = band_value
                                
                        for ch_band in eeg_band_columns:
                            row.append(channel_band_data[ch_band])
                                
                        writer.writerow(row)

print("Data successfully written to ml_input_output.csv")

Data successfully written to ml_input_output.csv
